In [29]:
# Locate project root dir, enable package imports from src/
import sys
from pathlib import Path

PROJ_ROOT = Path().resolve().parents[1]
sys.path.append(str(PROJ_ROOT / "src"))
print(PROJ_ROOT)

/Users/alexandre/Documents/GitHub/price_predictor


In [30]:
import os

# Load standardized paths and cleaning utilities
from project_utils.paths import RAW_DATA_DIR, CLEANED_DATA_DIR
import project_utils.cleaning_utils as cleaning_utils

import json

# Data handling and manipulation
import pandas as pd
import numpy as np

# Text processing and string matching
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [31]:
# Importing raw data
df_train = pd.read_csv(os.path.join(RAW_DATA_DIR, "train.csv"), index_col="carID")
df_test = pd.read_csv(os.path.join(RAW_DATA_DIR, "test.csv"), index_col="carID")

In [32]:
# Checking for duplicated unique car ID values
print("df_train:", df_train.index.is_unique)
print("df_test:", df_test.index.is_unique)

df_train: True
df_test: True


In [33]:
# df_train numerical features
numeric_features = df_train.select_dtypes(exclude="object").copy()
numeric_features.drop(columns="price", inplace=True)  # Drops target
# df_test numerical features
numeric_test = df_test.select_dtypes(exclude="object").copy()

In [34]:
numeric_features.describe().T

,count,mean,std,min,25%,50%,75%,max
year,74482.0,2017.096611,2.208704,1970.000000,2016.00,2017.0,2019.0,2024.121759
mileage,74510.0,23004.184088,22129.788366,-58540.574478,7423.25,17300.0,32427.5,323000.000000
tax,68069.0,120.329078,65.521176,-91.121630,125.00,145.0,145.0,580.000000
mpg,68047.0,55.152666,16.497837,-43.421768,46.30,54.3,62.8,470.800000
engineSize,74457.0,1.660136,0.573462,-0.103493,1.20,1.6,2.0,6.600000
paintQuality%,74449.0,64.590667,21.021065,1.638913,47.00,65.0,82.0,125.594308
previousOwners,74423.0,1.994580,1.472981,-2.345650,1.00,2.0,3.0,6.258371
hasDamage,74425.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000


An examination of the descriptive statistical summary of the training dataset reveals several noteworthy issues:
1. Skewed Distributions
Features such as 'mileage' and 'tax' exhibit substancial differences between their mean and median values, indicating pronounced skewness in their distributions. Furthermore, the target variable (price) also displays a similar pattern, suggesting that its distribution is likewise skewed.

2. Invalid Numerical Values
The features 'mileage', 'tax', 'mpg', 'engineSize' and 'previousOwners'  contain negative values, which are not physically meaningful according to the dataset’s metadata. These entries likely correspond to data entry errors or inconsistencies and therefore require correction or removal prior to downstream analysis.

3. Implausible Feature Ranges
The 'year' variable reaches a maximum of approximately 2024, despite the metadata indicating that vehicle information should only extend up to 2020. Similarly, the 'paintQuality' feature displays values up to approximately 125%, which is not realistic. These anomalies suggest further data quality issues that must be addressed.

4. Non-Informative Feature
The 'hasDamage' variable contains only a single observed value (0) across all records, indicating complete lack of variability.

The anomalies described in points 1 to 3 will be rectified as part of the present data-cleaning workflow. Whereas the issue highlighted in point 4 will be evaluated during the feature-selection phase.

In [35]:
numeric_test.describe().T

,count,mean,std,min,25%,50%,75%,max
year,31914.0,2017.102299,2.207969,1991.000000,2016.00,2017.0,2019.0,2024.121759
mileage,31878.0,22952.658921,22132.758713,-58540.574478,7298.25,17225.5,32500.0,279000.000000
tax,29259.0,120.569239,65.560570,-91.121630,125.00,145.0,145.0,580.000000
mpg,29279.0,55.210728,17.644635,-43.421768,46.30,54.3,62.8,470.800000
engineSize,31939.0,1.665377,0.574467,-0.103493,1.20,1.6,2.0,6.600000
paintQuality%,31942.0,64.446667,21.142188,1.638913,47.00,65.0,82.0,125.594308
previousOwners,31970.0,2.006118,1.472310,-2.345650,1.00,2.0,3.0,6.258371
hasDamage,31970.0,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000


Moreover, the provided test set exhibits the same data entry inconsistencies, indicating that these issues affect both splits of the dataset.

# Correcting invalid data entries

To resolve these issues, all invalid entries in both the training and test sets were corrected. In the case of the test set, the original row count had to be preserved to maintain index compatibility for downstream prediction submission; therefore, correction rather than removal was required.

In [36]:
# Correcting 'year'
# df_train
year_mask = numeric_features["year"] > 2020.0
numeric_features = numeric_features[~year_mask]

# df_test
test_year_mask = numeric_test["year"] > 2020
numeric_test["year"] = numeric_test["year"].mask(test_year_mask, other=2020.0)

In [37]:
# Correcting 'paintQuality%'
# df_train
paint_mask = numeric_features["paintQuality%"] > 100
numeric_features = numeric_features[~paint_mask]

# df_test
paint_test_mask = numeric_test["paintQuality%"] > 100
numeric_test["paintQuality%"] = numeric_test["paintQuality%"].mask(
    paint_test_mask, other=100
)

In [38]:
# Correcting negative values
# df_train
negative_mask = (numeric_features < 0).any(axis=1)
numeric_features = numeric_features[~negative_mask]

# df_test
test_negative_mask = numeric_test < 0
numeric_test[test_negative_mask] = numeric_test[test_negative_mask].abs()

In [39]:
print(
    f"Percentage of retained data after removing invalid entries: "
    f"{round(numeric_features.shape[0] / df_train.shape[0] * 100, 2)}%"
)

Percentage of retained data after removing invalid entries: 97.46%


In [40]:
# Applying corrections to DataFrames
df_train[numeric_features.columns] = numeric_features[numeric_features.columns]
df_test[numeric_test.columns] = numeric_test[numeric_test.columns]

## Categorical features

In [41]:
categories = df_train.select_dtypes(include="object").copy()
categories

,Brand,model,transmission,fuelType
carID,,,,
69512,VW,Golf,Semi-Auto,Petrol
53000,Toyota,Yaris,Manual,Petrol
6366,Audi,Q2,Semi-Auto,Petrol
29021,Ford,FIESTA,anual,Petrol
10062,BMW,2 Series,Manual,Petrol
...,...,...,...,...
37194,Mercedes,C Class,Manual,etrol
6265,Audi,Q3,Semi-Auto,Diesel
54886,Toyota,Aygo,Automatic,Petrol


A preliminary inspection of the data reveals several misspelled entries (e.g., Petrol, etrol). Before proceeding to model building and evaluation, these invalid textual entries must be corrected to ensure consistency across the categorical variables and to reduce the number of unique values within each feature.

To address these inconsistencies, we apply Python’s SequenceMatcher, which identifies the closest valid label for each misspelled value based on a similarity ratio computed between strings. Given two strings, the similarity ratio is defined as:
$$
ratio = \frac{2M}{T}
$$
+ Where:
    + $ M$ denotes the number of matching characters
    + $ T$ is the total number of characters in both strings

Despite its simplicity, the approach demonstrated strong performance in correcting minor typographical inconsistencies and ensuring a consistent representation of categorical text values throughout the dataset.

## Fixing brand names

In [42]:
categories.query("Brand.notnull()")["Brand"].unique()

array(['VW', 'Toyota', 'Audi', 'Ford', 'BMW', 'Skoda', 'Opel', 'Mercedes',
       'FOR', 'mercedes', 'Hyundai', 'w', 'ord', 'MW', 'bmw', 'yundai',
       'BM', 'Toyot', 'udi', 'Ope', 'AUDI', 'V', 'opel', 'pel', 'For',
       'pe', 'Mercede', 'audi', 'MERCEDES', 'OPEL', 'koda', 'FORD',
       'Hyunda', 'W', 'Aud', 'vw', 'hyundai', 'skoda', 'ford', 'TOYOTA',
       'ercedes', 'oyota', 'toyota', 'SKODA', 'Skod', 'HYUNDAI', 'kod',
       'v', 'for', 'SKOD', 'aud', 'KODA', 'PEL', 'yunda', 'or', 'UDI',
       'OYOTA', 'HYUNDA', 'mw', 'OPE', 'mercede', 'ERCEDES', 'ercede',
       'TOYOT', 'MERCEDE', 'ORD', 'ud', 'ope', 'AUD', 'hyunda', 'skod',
       'toyot'], dtype=object)

As suspected, several brand names are misspelled, and there are inconsistencies in the use of upper- and lowercase characters across the categorical variables. To address these issues, all categorical features were normalised to lowercase, thereby simplifying the correction of textual inconsistencies and ensuring uniformity throughout the dataset.

In [43]:
# Creates new DataFrame with normalized columns
corrected_category_df = categories.map(
    lambda x: np.nan if pd.isna(x) else str(x).strip().lower()
).copy()

In [44]:
valid_brands = [
    "toyota",
    "mercedes",
    "vw",
    "audi",
    "ford",
    "bmw",
    "opel",
    "hyundai",
    "skoda",
]

# Correct brand entries
corrected_category_df["Brand"] = corrected_category_df["Brand"].apply(
    lambda brand: cleaning_utils.category_analyzer(brand, valid_brands)
)


In [45]:
corrected_category_df.groupby("Brand")["model"].unique()

Brand
audi        [q2, a3, q3, q5, tt, a1, a8, a4, a5, a6, q7, q...
bmw         [2 series, 3 series, 5 series, 4 series, x3, x...
ford        [fiesta, focus, ka+, ecosport, ecospor, nan, b...
hyundai     [i30, tucson, ix20, i20, kona, i10, ioniq, san...
mercedes    [a clas, a class, glc class, c class, e class,...
opel        [insignia, mokka x, corsa, astra, viva, adam, ...
skoda       [octavia, fabia, rapid, yeti outdoor, scala, k...
toyota      [yaris, c-hr, aygo, auris, rav4, prius, coroll...
vw          [golf, passat, polo, up, sharan, tiguan, t-roc...
Name: model, dtype: object

## Fixing model names


In [46]:
# Dictionary of 'model' per 'brand' (sorted by frequency)
sort_by_freq = (
    corrected_category_df.groupby("Brand")["model"]
    .value_counts()
    .groupby(level=0)
    .apply(lambda s: s.index.get_level_values("model").tolist())
    .to_dict()
)

To further reduce textual inconsistencies in the model feature, an automated correction procedure was implemented using character-level TF–IDF embeddings and cosine similarity. This approach allows misspelled or near-duplicate model names to be mapped to their most common, standardised counterparts within each brand.

In [47]:
# Contants
NGRAM = (2, 3)  # Character n-gram range used for TF-IDF encoding
THR = 0.8  # Minimum cosine similarity required to assign a correction
K = 18  # Number of top-frequency model names used as anchor references

correct_dict = {}

for brand in valid_brands:
    # Stores model names sorted by frequency and vecotrizes them
    uniq = np.array(sort_by_freq[brand], dtype=object)
    vec = TfidfVectorizer(analyzer="char", ngram_range=NGRAM)
    X = vec.fit_transform(uniq)

    # Vectorize the top-K most frequent model names (anchors)
    A = vec.transform(uniq[: min(K, len(uniq))])

    # Comput cosine similarity between each model anme and the anchor set
    S = cosine_similarity(X, A)

    # Identify the most similar anchor for each 'model'
    best_idx = S.argmax(axis=1)
    best_sim = S.max(axis=1)

    # Assign corrected labels where similarity exceeds the threshold
    assigned = np.where(best_sim >= THR, uniq[best_idx], uniq)

    # Store corrected model names for the current brand
    correct_dict[brand] = assigned

# Applies 'model' label corrections
corrected_category_df["model"] = cleaning_utils.model_cleaner(
    corrected_category_df, correct_dict, valid_brands
)

In [48]:
# List of corrected models per brand
for brand in valid_brands:
    print(f"{brand}:")
    print(
        corrected_category_df.query("Brand == @brand")["model"].unique(),
        "\n" + "-" * 50,
    )

toyota:
['yaris' 'c-hr' 'aygo' 'auris' 'rav4' 'prius' 'corolla' nan 'verso' 'ayg'
 'gt86' 'land cruiser' 'proace verso' 'hilux' 'avensis' 'camry' 'yari'
 'supra' 'c-h' 'verso-s' 'auri' 'iq' 'rav' 'urban cruiser' 'urban cruise'] 
--------------------------------------------------
mercedes:
['a class' 'glc class' 'c class' 'e class' 'c clas' 'slk' 'x-class'
 'cl class' 'gls class' 'gle class' 's class' 'gla class' 'sl class'
 'b class' 'v class' nan 'cls class' 'cla class' 'gl class' 'm class'
 'clk' 'g class' 'glb class' '200' '220' '230' 'm clas' 'sl' 'clc class'] 
--------------------------------------------------
vw:
['golf' 'passat' 'polo' 'up' 'sharan' 'tiguan' 't-roc' 'arteon' nan
 'touran' 'caddy maxi life' 'beetle' 'touareg' 'shuttle' 'tiguan allspace'
 'caravelle' 'pol' 't-cross' 'golf sv' 'gol' 'california' 'caddy life'
 'scirocco' 'amarok' 'cc' 'eos' 'jetta' 'u' 'caddy maxi' 'caddy' 'beetl'
 'fox' 'caddy maxi lif'] 
--------------------------------------------------
audi:
['q

Since some model names remained unstandardised (e.g., gol instead of golf), a dictionary of corrected model labels was employed to resolve these residual inconsistencies.

In [49]:
cleaned_dict_dir = os.path.join(
    PROJ_ROOT, "src", "project_utils", "cleaned_models.json"
)

with open(cleaned_dict_dir, "r") as f:
    brand_models_dict = json.load(f)
corrected_category_df["model"] = cleaning_utils.model_cleaner(
    corrected_category_df, brand_models_dict, valid_brands
)

In [50]:
# List of corrected models per brand
for brand in valid_brands:
    print(f"{brand}:")
    print(
        corrected_category_df.query("Brand == @brand")["model"].unique(),
        "\n" + "-" * 50,
    )

toyota:
['yaris' 'c-hr' 'aygo' 'auris' 'rav4' 'prius' 'corolla' nan 'verso' 'gt86'
 'land cruiser' 'proace verso' 'hilux' 'avensis' 'camry' 'supra' 'verso-s'
 'iq' 'urban cruiser'] 
--------------------------------------------------
mercedes:
['a class' 'glc class' 'c class' 'e class' 'slk' 'x-class' 'cl class'
 'gls class' 'gle class' 's class' 'gla class' 'sl class' 'b class'
 'v class' nan 'cls class' 'cla class' 'gl class' 'm class' 'clk'
 'g class' 'glb class' 'clc class'] 
--------------------------------------------------
vw:
['golf' 'passat' 'polo' 'up' 'sharan' 'tiguan' 't-roc' 'arteon' nan
 'touran' 'caddy maxi life' 'beetle' 'touareg' 'shuttle' 'caravelle'
 't-cross' 'golf sv' 'california' 'caddy life' 'scirocco' 'amarok' 'cc'
 'eos' 'jetta' 'caddy maxi' 'caddy'] 
--------------------------------------------------
audi:
['q2' 'a3' 'q3' 'q5' 'tt' 'a1' 'a8' 'a4' 'a5' 'a6' 'q7' 'q8' nan 'rs6'
 'a7' 'rs5' 'r8' 'rs3' 'sq5' 'rs4' 'sq7' 's3' 's4' 'a2' 's8' 's5'] 
------------------

## Accessing remaining categorical features

In [51]:
np.sort(corrected_category_df.query("transmission.notnull()")["transmission"].unique())

array(['anua', 'anual', 'automati', 'automatic', 'emi-aut', 'emi-auto',
       'manua', 'manual', 'nknow', 'nknown', 'other', 'semi-aut',
       'semi-auto', 'unknow', 'unknown', 'utomati', 'utomatic'],
      dtype=object)

In [52]:
np.sort(corrected_category_df.query("fuelType.notnull()")["fuelType"].unique())

array(['diese', 'diesel', 'electric', 'etro', 'etrol', 'hybri', 'hybrid',
       'iese', 'iesel', 'othe', 'other', 'petro', 'petrol', 'ther',
       'ybri', 'ybrid'], dtype=object)

In [53]:
# Lists of valid categories
valid_transmission = ["manual", "semi-auto", "automatic", "other", "unknown"]
valid_fuelTypes = ["petrol", "diesel", "hybrid", "electric", "other"]

# Apply correction to 'transmission'
corrected_category_df["transmission"] = corrected_category_df["transmission"].apply(
    lambda val: cleaning_utils.category_analyzer(val, valid_transmission)
)
# Apply correction to 'fuelTypes'
corrected_category_df["fuelType"] = corrected_category_df["fuelType"].apply(
    lambda val: cleaning_utils.category_analyzer(val, valid_fuelTypes)
)

In [54]:
# Replacing columns on original df with cleaned ones
df_train[categories.columns] = corrected_category_df[categories.columns]

### Test set categories

In [55]:
# Normalizes the test df categoricals
normalized_df_test = df_test.select_dtypes(include="object").copy()
normalized_df_test = normalized_df_test.map(
    lambda x: np.nan if pd.isna(x) else str(x).strip().lower()
)

##-- Data cleaning --##

# 'Brand' column
normalized_df_test["Brand"] = normalized_df_test["Brand"].apply(
    lambda brand: cleaning_utils.category_analyzer(brand, valid_brands)
)

# 'model' column
normalized_df_test["model"] = cleaning_utils.model_cleaner(
    normalized_df_test, brand_models_dict, valid_brands
)

# 'transmission' column
normalized_df_test["transmission"] = normalized_df_test["transmission"].apply(
    lambda val: cleaning_utils.category_analyzer(val, valid_transmission)
)

# 'fuelType' column
normalized_df_test["fuelType"] = normalized_df_test["fuelType"].apply(
    lambda val: cleaning_utils.category_analyzer(val, valid_fuelTypes)
)

# Replacing corrected columns on the df_test
df_test[normalized_df_test.columns] = normalized_df_test[normalized_df_test.columns]


In [56]:
df_train.to_csv(os.path.join(CLEANED_DATA_DIR, "cleaned_train_data.csv"))
df_test.to_csv(os.path.join(CLEANED_DATA_DIR, "cleaned_test_data.csv"))